In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

In [3]:
import tensorflow as tf
from tensorflow.keras.regularizers import L2
import random as python_random
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Embedding, Dropout, Dense, Flatten, Concatenate, Add, MultiHeadAttention, LeakyReLU, Multiply, Layer, Attention, GRU, Bidirectional, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import LabelEncoder
import re, os
import pandas as pd
import numpy as np

In [4]:
# GPU 사용
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
from tensorflow.python.client import device_lib
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
def reset_random_seeds():
    tf.random.set_seed(42)
    np.random.seed(42)
    python_random.seed(42)
reset_random_seeds()

# 데이터 로드

In [7]:
base_df = pd.read_pickle('/content/drive/MyDrive/Jeong(정의주 논문)/Lee(이청용논문)/1_Dataset/Cell_Phones_and_Accessories_Average.pkl')
df = base_df
df

,reviewerID,asin,summary,reviewText,image,vote,overall,bert,roberta,VGG16
0,AAKKVN6IJJKO1,098949232X,Keep looking up at night.,This is one of the best calendars that you can...,https://images-na.ssl-images-amazon.com/images...,NaN,5.0,"[-0.0984451100230217, -0.00199960358440876, 0....","[-0.07951568067073822, 0.08175838738679886, -0...","[0.0, 0.0, 0.88047546, 0.0, 0.0, 0.0, 0.0, 0.5..."
1,A3N778P1L4YH9Y,7391002801,Case is cheaply made. If you aren't using an ...,Case is cheaply made. If you aren't using an a...,https://images-na.ssl-images-amazon.com/images...,2,1.0,"[-0.6160225868225098, 0.25027498602867126, -0....","[-0.10782118886709213, 0.11555437743663788, -0...","[0.0, 0.0, 0.0, 0.22367102, 0.7665119, 0.0, 0...."
2,A34GGU76V8AE5T,7887421268,"My case looks used, could have been much bette...",I was very excited to find a really cute cover...,https://images-na.ssl-images-amazon.com/images...,19,2.0,"[-0.051112815737724304, -0.1525549292564392, -...","[-0.06908304989337921, 0.10518966615200043, -0...","[0.0, 0.0, 0.3041149, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,A2FZ0P2S6KCJ5P,8288862993,"Fake, it's not a real Samsung product, bad qua...","I purchased this car charger ""Samsung Official...",https://images-na.ssl-images-amazon.com/images...,3,1.0,"[0.04953824356198311, -0.35632771253585815, 0....","[-0.07209423184394836, 0.13247931003570557, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,A2G6OURH28WI8R,9638762632,Give me the POOOWWWAAAA!!!,The PowerBear HTC One M9 Battery Case is absol...,https://images-na.ssl-images-amazon.com/images...,NaN,4.0,"[-0.5801865458488464, -0.5791118741035461, -0....","[-0.08062966167926788, 0.14100584387779236, -0...","[0.0, 0.0, 0.0, 0.26141828, 0.0, 0.0, 0.317899..."
...,...,...,...,...,...,...,...,...,...,...
179244,A3BMH52AZICG58,B01HJBS5C2,Protection that works,Dropped my phone many times and this case did ...,https://images-na.ssl-images-amazon.com/images...,NaN,5.0,"[-0.24131955206394196, -0.0698041170835495, -0...","[-0.08227464556694031, 0.13271795213222504, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
179245,A1DEBIPRY76UHQ,B01HJCR7KW,pieces starting falling off 2 days i started u...,pieces starting falling off 2 days i started ...,https://images-na.ssl-images-amazon.com/images...,NaN,1.0,"[-0.3720323145389557, 0.13811653852462769, 0.7...","[-0.08569245785474777, 0.05454415827989578, -0...","[0.34200567, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0..."
179246,AMUEAMKB4E33M,B01HJC7N4C,Get more out of your smartphone .......,Super Fun! The RockRoc 3d vr headset is waaaay...,https://images-na.ssl-images-amazon.com/images...,NaN,5.0,"[-0.20503820478916168, -0.5619103908538818, 0....","[-0.07647551596164703, 0.09452658146619797, -0...","[0.0, 0.16607133, 0.0, 0.0, 0.17669825, 1.0349..."
179247,A204LEYV64Y3VW,B01HJF9OI2,I really hate complaining and bad reviews,I had it for a whole 8 days and for the first ...,https://images-na.ssl-images-amazon.com/images...,NaN,1.0,"[-0.11504127085208893, -0.17505033314228058, 0...","[-0.09449812769889832, 0.13758613169193268, -0...","[0.0, 0.07752671, 0.0, 0.05892245, 0.0, 0.0, 0..."


In [8]:
# Encode user and item IDs
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

df['reviewerID'] = user_encoder.fit_transform(df['reviewerID'])
df['asin'] = item_encoder.fit_transform(df['asin'])

In [9]:
# Count unique users and items
user_num = df['reviewerID'].nunique()
item_num = df['asin'].nunique()

print(f'Unique Users: {user_num}')
print(f'Unique Items: {item_num}')

Unique Users: 148405
Unique Items: 60665


In [10]:
df = base_df.dropna(subset = ['VGG16'], axis=0)
df.isna().sum()

reviewerID         0
asin               0
summary          191
reviewText         0
image              0
vote          114572
overall            0
bert               0
roberta            0
VGG16              0
dtype: int64

In [11]:
# Split the dataset
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# traing dataset
train_users, train_items = train_data[['reviewerID']].values, train_data[['asin']].values
train_roberta = np.array(train_data['roberta'].tolist())
train_image = np.array(train_data['VGG16'].tolist())
train_ratings = train_data['overall'].values

# test dataset
test_users, test_items = test_data[['reviewerID']].values, test_data[['asin']].values
test_roberta = np.array(test_data['roberta'].tolist())
test_image = np.array(test_data['VGG16'].tolist())
test_ratings = test_data['overall'].values

In [12]:
# Define feed-forward network
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff),
      LeakyReLU(alpha=0.01),
      tf.keras.layers.Dense(d_model),
      LeakyReLU(alpha=0.01)
  ])

In [13]:
# Adjust the CoAttentionBlock
class CoAttentionBlock(tf.keras.layers.Layer):
    def __init__(self, num_heads, key_dim, dff, dropout_rate=0.1, epsilon=1e-6, **kwargs):
        super(CoAttentionBlock, self).__init__(**kwargs)
        self.multi_head_attention = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)
        self.dropout = Dropout(dropout_rate)
        self.add = Add()
        self.layer_norm = tf.keras.layers.LayerNormalization(epsilon = epsilon)
        self.ffn = point_wise_feed_forward_network(key_dim, dff)
        self.flatten = Flatten()


    def call(self, inputs):
        text_expanded, image_expanded = inputs
        # Attention from text to image features
        co_attention = self.multi_head_attention(text_expanded, image_expanded, image_expanded)
        co_attention = self.dropout(co_attention)
        co_attention = self.add([co_attention, text_expanded])
        co_attention = self.layer_norm(co_attention)

        # Feed-forward network
        co_attention_ffn = self.ffn(co_attention)
        co_attention_ffn = self.dropout(co_attention_ffn)
        co_attention_ffn = self.add([co_attention_ffn, co_attention])
        co_attention_ffn = self.layer_norm(co_attention_ffn)

        return self.flatten(co_attention_ffn)

In [14]:
def EJEJ(user_num, item_num, feature_dims, num_heads, dropout_rate, learning_rate):
        user_input = Input(shape=(1,), dtype='int32', name='UserInput')
        user_embedding = Embedding(user_num, feature_dims, input_length=user_input.shape[1], embeddings_regularizer=L2(), name='UserIDEmb')(user_input)
        user_embedding = Flatten(name='UserFlatten')(user_embedding)

        # Item Input
        item_input = Input(shape=(1,), dtype='int32', name='ItemInput')
        item_embedding = Embedding(item_num, feature_dims, input_length=item_input.shape[1], embeddings_regularizer=L2(), name='ItemIDEmb')(item_input)
        item_embedding = Flatten(name='itemFlatten')(item_embedding)

        # Concatenate User and Item Embeddings
        user_item_mlp = Concatenate(name="UserItemConcat")([user_embedding, item_embedding])
        user_item_mlp = Dropout(rate=dropout_rate)(user_item_mlp)

        # Define MLP layer sizes based on feature_dims
        if feature_dims == 32:
            mlp_layer_sizes = [32, 16, 8]
        elif feature_dims == 64:
            mlp_layer_sizes = [64, 32, 16]
        elif feature_dims == 128:
            mlp_layer_sizes = [128, 64, 32]
        elif feature_dims == 256:
            mlp_layer_sizes = [256, 128, 64]
        else:
            raise ValueError("Unsupported predictive factors size.")

        # MLP Layers for User and Item
        for idx, size in enumerate(mlp_layer_sizes):
            user_item_mlp = Dense(units=size, name=f'Dense_User_Item_{idx+1}')(user_item_mlp)
            user_item_mlp = LeakyReLU(alpha=0.01)(user_item_mlp)
            if idx < len(mlp_layer_sizes) - 1:
                user_item_mlp = Dropout(rate=dropout_rate)(user_item_mlp)

        # RoBERTa Inputs
        roberta = Input(shape=(768,), name='RoBERTaInput')
        roberta_MLP = Dense(units=256, name='roberta_MLP')(roberta)
        roberta_MLP = LeakyReLU(alpha=0.01)(roberta_MLP)
        roberta_MLP = Dropout(rate=dropout_rate)(roberta_MLP)

        # Image Input and MLP Layers
        input_image = Input(shape=(4096,), name='ImageInput')
        input_image_dense = Dense(units=2048, name='Dense_Image_1')(input_image)
        input_image_dense = LeakyReLU(alpha=0.01)(input_image_dense)
        input_image_dense = Dropout(rate=dropout_rate)(input_image_dense)
        input_image_dense = Dense(units=1024, name='Dense_Image_2')(input_image_dense)
        input_image_dense = LeakyReLU(alpha=0.01)(input_image_dense)
        input_image_dense = Dropout(rate=dropout_rate)(input_image_dense)
        input_image_dense = Dense(units=256, name='Dense_Image_3')(input_image_dense)
        input_image_dense = LeakyReLU(alpha=0.01)(input_image_dense)
        input_image_dense = Dropout(rate=dropout_rate)(input_image_dense)

        # Co-Attention Mechanism
        text_expanded = tf.keras.layers.LayerNormalization(epsilon=1e-6)(roberta_MLP)
        image_expanded = tf.keras.layers.LayerNormalization(epsilon=1e-6)(input_image_dense)

        text_expanded = tf.expand_dims(text_expanded, axis=1)
        image_expanded  = tf.expand_dims(image_expanded, axis=1)

        co_attention_block = CoAttentionBlock(num_heads=num_heads, key_dim=256, dff=2048, dropout_rate=0.1, epsilon=1e-6)

        co_attention_text_image = co_attention_block((text_expanded, image_expanded))
        co_attention_text_image = Dense(units=int(mlp_layer_sizes[-1]), name='Dense_CoAttention_Text_Image')(co_attention_text_image)
        co_attention_text_image = LeakyReLU(alpha=0.01)(co_attention_text_image)
        co_attention_text_image = Dropout(rate=dropout_rate)(co_attention_text_image)

        co_attention_image_text = co_attention_block((image_expanded, text_expanded))
        co_attention_image_text = Dense(units=int(mlp_layer_sizes[-1]), name='Dense_CoAttention_Image_Text')(co_attention_image_text)
        co_attention_image_text = LeakyReLU(alpha=0.01)(co_attention_image_text)
        co_attention_image_text = Dropout(rate=dropout_rate)(co_attention_image_text)

        # Concatenate text-image
        review_image_concat = Concatenate(name='Text_Image_concat')([co_attention_text_image, co_attention_image_text])

        # Concatenate User-Item and text-image
        user_item_review_image = Concatenate(name='User_Item_Text_Factor')([user_item_mlp, review_image_concat])

        # MLP Layers for User-Item and text-image
        user_item_review_image = Dense(units=int(mlp_layer_sizes[-1]), name='User_Item_Review_Image')(user_item_review_image)
        user_item_review_image = LeakyReLU(alpha=0.01)(user_item_review_image)
        user_item_review_image_dropout = Dropout(rate=dropout_rate)(user_item_review_image)

        # Compile Model
        outputs = Dense(1, activation='linear', name='outputs')(user_item_review_image_dropout)

        # Model
        model = Model(inputs=[user_input, item_input, roberta, input_image], outputs=outputs)
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate), loss = "mse", metrics = ["mse", "mae"])

        return model


In [15]:
# Configuration of EarlyStopping
es = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience = 5, restore_best_weights = True)

In [23]:
# Parameter setting
dropout_rate = 0.1
feature_dims = 128
num_heads = 6
learning_rate = 0.001
batch_size= 256

# Train the model
proposed_model = EJEJ(user_num, item_num, feature_dims, num_heads, dropout_rate, learning_rate)

# Model training with EarlyStopping
proposed_model.fit([train_users, train_items, train_roberta, train_image], train_ratings,
                   batch_size=batch_size, epochs=50, validation_split=0.125, callbacks=[es])

Epoch 1/50
490/490 [==============================] - 21s 27ms/step - loss: 7.9078 - mse: 1.4671 - mae: 0.9152 - val_loss: 0.8336 - val_mse: 0.6755 - val_mae: 0.5407
Epoch 2/50
490/490 [==============================] - 12s 25ms/step - loss: 1.0112 - mse: 0.8025 - mae: 0.6541 - val_loss: 0.8156 - val_mse: 0.6196 - val_mae: 0.5196
Epoch 3/50
490/490 [==============================] - 12s 25ms/step - loss: 0.9376 - mse: 0.7335 - mae: 0.6167 - val_loss: 0.7804 - val_mse: 0.6128 - val_mae: 0.5444
Epoch 4/50
490/490 [==============================] - 12s 24ms/step - loss: 0.9245 - mse: 0.6587 - mae: 0.5840 - val_loss: 0.8229 - val_mse: 0.5950 - val_mae: 0.5675
Epoch 5/50
490/490 [==============================] - 12s 24ms/step - loss: 1.1043 - mse: 0.5492 - mae: 0.5374 - val_loss: 1.1504 - val_mse: 0.6642 - val_mae: 0.5922
Epoch 6/50
490/490 [==============================] - 12s 24ms/step - loss: 0.9559 - mse: 0.4061 - mae: 0.4571 - val_loss: 1.5807 - val_mse: 0.7657 - val_mae: 0.5847
Epoc

In [24]:
# Evaluate the model
predicted_ratings = proposed_model.predict([test_users, test_items, test_roberta, test_image])

# Calculate MAE
MAE = mean_absolute_error(test_ratings, predicted_ratings)
print(f"MAE: {MAE:.3f}")

# Calculate MSE
MSE = mean_squared_error(test_ratings, predicted_ratings)
print(f"MSE: {MSE:.3f}")

# Calculate RMSE
RMSE = np.sqrt(MSE)
print(f"RMSE: {RMSE:.3f}")

# Calculate MAPE
MAPE = mean_absolute_percentage_error(test_ratings, predicted_ratings) * 100
print(f"MAPE: {MAPE:.3f}%")

1120/1120 [==============================] - 5s 4ms/step
MAE: 0.543
MSE: 0.611
RMSE: 0.782
MAPE: 21.189%


In [25]:
# ouput
print( MAE, "\n", MSE, "\n", RMSE, "\n", MAPE )

0.543300255936727 
 0.6114422662798202 
 0.7819477388418105 
 21.188524579424424
